In [51]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import *
import pandas as pd
import cmath as cm

In [52]:
#Universal Constants
h=6.6260700e-34             #Planck's Constant
kb=1.38064852e-23           #Boltzmann's Constant
c=2.998e8;                  #Light Velocity in vacuum
q=1.602176565e-19;          #Electron Charge
Ev=q;                       #Electronvolt
em=9.1093837015e-31;        #Electron Mass
Phi0=h/(2*q);               #Quantum Magnetic Flux

In [53]:
#TES parameters
Top=300e-3;     #Operating Temperature 
G=70e-12;       #G W/K
C=500e-12;        #Thermal Capacitance
Res=1100000;        #TES respositivity with ETF A/W
Mctes=50e-12;      #Inductive Coupling to SQUID

In [54]:
# SQUID Parameters 
Lj=60e-12;               #(Josepson inductance Lj=h_bar/2eIc)       
Mc=1.65e-12;                #Screening Mutual Inductance

In [55]:
# Parameters Resonator
f1=6e9;               #Unperturbed Resonance Frequency
Z1=50;                #Resonator Impedance
Cc=1e-15;             #Coupling Capacitance
Attdb=20;             #Resonator Attenuation

In [56]:
# Readout Parameters
N=400                    #tones 
fmax=8e9;                  #final frequency
fmin=4e9;                  #Initial frequency
Spacing=(fmax-fmin)/N    #Resonator Spacing 
BW=4e3;                  #Readout Bandwidth resonator
Prdbm=-60;               #Readout Power per pixel

In [57]:
#NEP to flux spectral density
NEP=1.7e-17;
PSDi=NEP*Res;                        #Current Spectal density due to CMB noise on TES. 

FSD=PSDi*Mctes;                     #Flux Spectral density refered to SQUD due to CMB signal on TES 
FSDu=FSD/Phi0/1e-6;                   

#Print Params
print("FSD=%7.3e uPhi0/sqrt(Hz)" %FSDu, "PDSi=%7.3e A/sqrt(Hz)" %PSDi)

FSD=4.522e-01 uPhi0/sqrt(Hz) PDSi=1.870e-11 A/sqrt(Hz)


In [58]:
#LNA Noise Temperature to NEP
Te=30;                                   #Noise Temperature
NEPcalc=np.sqrt(4*kb*Lj*Te/(np.pi*f1))/Mctes/Res; 

#print Params
print("NEPcalc=%7.3eW/sqrt(Hz)" %NEPcalc)

NEPcalc=4.175e-17W/sqrt(Hz)


In [59]:
#LNA Noise Temperature from FDS
Tn=(FSD**2)*np.pi*f1/(4*kb*Lj)

#Print Params
print("Tn=%7.3e K" %Tn)

Tn=4.973e+00 K


In [60]:
# Readout SNR per pixel

Pout=(1e-3)*10**((Prdbm-Attdb)/10);            #Output Signal Power
Nout=kb*Tn*BW;                                 #Output Noise

#SNR Calculation
SNR=10*np.log10(Pout/Nout)

#Print Params
print("SNR=%3.2f dB" %SNR)

SNR=75.61 dB


In [61]:
# RF Power and voltage in lines after uMUX
Ptot=Prdbm-Attdb+10*np.log10(N)

Armst=10**((Ptot-10)/20)/np.sqrt(2);            #Output Signal Power
Amaxt=4*Armst;

#Print Params
print("Ptot=%3.1f dBm" %Ptot, "Arms=%3.2e V" %Armst, "Amaxt=%3.2e V" %Amaxt)

Ptot=-54.0 dBm Arms=4.47e-04 V Amaxt=1.79e-03 V


In [62]:
# RF Power and voltage in lines after uMUX
Glna=36;
Ptot=Prdbm-Attdb+10*np.log10(N)+Glna

Armst=10**((Ptot-10)/20)/np.sqrt(2);            #Output Signal Power
Amaxt=4*Armst;

#Print Params
print("Ptot=%3.1f dBm" %Ptot, "Arms=%3.2e V" %Armst, "Amaxt=%3.2e V" %Amaxt)

Ptot=-18.0 dBm Arms=2.82e-02 V Amaxt=1.13e-01 V


In [63]:
#RF Rx Chain parameters dB
NFlna=10*np.log10(1+Tn/300)                   #LNA noise Figure
NFdb=np.array([NFlna,10.0,10.0,10.0,10.0])    #Noise Figure dB 
Gdb=np.array([36,10,-5,10,-5])                #Gain dB
IIP3db=np.array([-44,-2,-2,-2,-2])               #Outout IP3 dB

#RF Rx Chain parameters conversion
NF=10**(NFdb/10)                   #Noise Figure
G=10**(Gdb/10)                     #Gain
IIP3=(1e-3)*10**(IIP3db/10)        #IIP3

#NF and IP3 calculation
NFt=NF[0]+(NF[1]-1)/G[0]+(NF[2]-1)/(G[0]*G[1])+(NF[3]-1)/(G[0]*G[1]*G[2])+(NF[4]-1)/(G[0]*G[1]*G[2]*G[3])

NFtdb=10*np.log10(NFt)

print("NFt=%3.5f" %NFt,"NFtdb=%3.3f dB" %NFtdb)

NFt=1.01985 NFtdb=0.085 dB


In [137]:
#Noise Figure chain function
def noisefig(NF,G):
    Gp=G
    NFt=NF[0]
    for i in range(1,len(NF)):
        Gp[i]=Gp[i-1]*G[i]
    for k in range(1,len(NF)-1):
        NFt+=(NF[k]-1)/Gp[k-1]
    return NFt

NFtf=noisefig(NF,G)
NFtfdb=10*np.log10(NFtf)

print("NFtf=%3.5f" %NFtf,"NFtfdb=%3.3f" %NFtfdb)

NFtf=1.01884 NFtfdb=0.081


In [147]:
#IP3 chain function
def noiseip(P3,G):
    ip3p=P3
    for i in range(1,len(P3)):
        ip3p[i]=1/(1/(ip3p[i-1]*G[i])+1/P3[i])
    return ip3p[2]
                
IIP3t=noiseip([1.,1.,1],[1.,1.,1])
IIP3tdb=10*np.log10(IIP3t)

print("IIP3t=%3.5f" %IIP3t,"IIP3tdb=%3.5f" %IIP3tdb)

IIP3t=0.33333 IIP3tdb=-4.77121


In [109]:
len([1,1])

2

In [144]:
ip3p=[1.,1.]
G=[1.,1.]
P3=ip3p;
for i in range(1,len(P3)):
        ip3p[i]=1/(1/(ip3p[i-1]*G[i])+1/P3[i])

print(ip3p)

[1.0, 0.5]
